In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Quantum approximate optimization algorithm

*Tinatayang paggamit: 22 minuto sa Heron r3 processor (TANDAAN: Ito ay isang tantiya lamang. Maaaring mag-iba ang inyong runtime.)*
## Background
Ipinakikita ng tutorial na ito kung paano ipatupad ang **Quantum Approximate Optimization Algorithm (QAOA)** – isang hybrid (quantum-classical) iterative method – sa konteksto ng Qiskit patterns. Lulutasin mo muna ang **Maximum-Cut** (o **Max-Cut**) problem para sa isang maliit na graph at pagkatapos ay matututo kung paano ito isagawa sa utility scale. Lahat ng hardware executions sa tutorial ay dapat tumakbo sa loob ng time limit para sa libreng-maaccessible na Open Plan.

Ang Max-Cut problem ay isang optimization problem na mahirap lutasin (mas tiyak, ito ay isang *NP-hard* problem) na may ilang iba't ibang aplikasyon sa clustering, network science, at statistical physics. Isinasaalang-alang ng tutorial na ito ang isang graph ng mga node na konektado ng mga edge, at naglalayong hatiin ang mga node sa dalawang set nang sa gayon ay ang bilang ng mga edge na tinatahak ng cut na ito ay pinakamataas.

![Illustration of a max-cut problem](../docs/images/tutorials/quantum-approximate-optimization-algorithm/maxcut-illustration.avif)
## Requirements
Bago magsimula ng tutorial na ito, siguraduhing mayroon kayong sumusunod na naka-install:
- Qiskit SDK v1.0 o mas bago, na may [visualization](https://docs.quantum.ibm.com/api/qiskit/visualization) support
- Qiskit Runtime v0.22 o mas bago (`pip install qiskit-ibm-runtime`)

Bilang karagdagan, kakailanganin ninyo ng access sa isang instance sa [IBM Quantum Platform](/guides/cloud-setup). Tandaan na ang tutorial na ito ay hindi maaaring isagawa sa Open Plan, dahil nagpapatakbo ito ng workloads gamit ang [sessions](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/session), na available lamang sa Premium Plan access.
## Setup

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import rustworkx as rx
from rustworkx.visualization import mpl_draw as draw_graph
import numpy as np
from scipy.optimize import minimize
from collections import defaultdict
from typing import Sequence


from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import QAOAAnsatz
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator
from qiskit_ibm_runtime import SamplerV2 as Sampler

## Part I. Small-scale QAOA
Ang unang bahagi ng tutorial na ito ay gumagamit ng isang small-scale Max-Cut problem upang ilarawan ang mga hakbang upang malutas ang isang optimization problem gamit ang quantum computer.

Upang magbigay ng konteksto bago i-map ang problemang ito sa isang quantum algorithm, mas mauunawaan ninyo kung paano ang Max-Cut problem ay nagiging isang classical combinatorial optimization problem sa pamamagitan ng pag-consider muna sa minimization ng isang function na $f(x)$

$$
\min_{x\in {0, 1}^n}f(x),
$$

kung saan ang input na $x$ ay isang vector na ang mga component ay tumutugma sa bawat node ng isang graph. Pagkatapos, limitahan ang bawat isa sa mga component na ito upang maging alinman sa $0$ o $1$ (na kumakatawan sa pagsasama o hindi pagsasama sa cut). Ang small-scale example case na ito ay gumagamit ng isang graph na may $n=5$ nodes.

Maaari kayong sumulat ng function ng isang pares ng nodes na $i,j$ na nagsasaad kung ang kaukulang edge na $(i,j)$ ay nasa cut. Halimbawa, ang function na $x_i + x_j - 2 x_i x_j$ ay 1 lamang kung isa sa alinman sa $x_i$ o $x_j$ ay 1 (na nangangahulugang ang edge ay nasa cut) at zero kung hindi. Ang problema ng pag-maximize sa mga edge sa cut ay maaaring iformula bilang

$$
\max_{x\in {0, 1}^n} \sum_{(i,j)} x_i + x_j - 2 x_i x_j,
$$

na maaaring isulat muli bilang minimization sa anyo ng

$$
\min_{x\in {0, 1}^n} \sum_{(i,j)}  2 x_i x_j - x_i - x_j.
$$

Ang minimum ng $f(x)$ sa kasong ito ay kapag ang bilang ng mga edge na tinatahak ng cut ay pinakamataas. Tulad ng inyong makikita, walang kaugnayan sa quantum computing pa. Kailangan ninyong i-reformulate ang problemang ito sa isang bagay na mauunawaan ng quantum computer.
Simulan ang inyong problema sa pamamagitan ng paglikha ng isang graph na may $n=5$ nodes.

In [2]:
n = 5

graph = rx.PyGraph()
graph.add_nodes_from(np.arange(0, n, 1))
edge_list = [
    (0, 1, 1.0),
    (0, 2, 1.0),
    (0, 4, 1.0),
    (1, 2, 1.0),
    (2, 3, 1.0),
    (3, 4, 1.0),
]
graph.add_edges_from(edge_list)
draw_graph(graph, node_size=600, with_labels=True)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/6ced6bea-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/6ced6bea-0.avif)

### Step 1: Map classical inputs to a quantum problem
Ang unang hakbang ng pattern ay ang pag-map ng classical problem (graph) sa mga quantum **circuits** at **operators**. Upang gawin ito, may tatlong pangunahing hakbang na dapat gawin:

1. Gumamit ng serye ng mathematical reformulations, upang katawanin ang problemang ito gamit ang Quadratic Unconstrained Binary Optimization (QUBO) problems notation.
2. Isulat muli ang optimization problem bilang isang Hamiltonian kung saan ang ground state ay tumutugma sa solusyon na nagpapaliit sa cost function.
3. Lumikha ng quantum circuit na maghahanda sa ground state ng Hamiltonian na ito sa pamamagitan ng prosesong katulad ng quantum annealing.

**Tandaan:** Sa QAOA methodology, sa huli ay gusto ninyong magkaroon ng operator (**Hamiltonian**) na kumakatawan sa **cost function** ng ating hybrid algorithm, pati na rin ang isang parametrized circuit (**Ansatz**) na kumakatawan sa mga quantum states na may mga kandidatong solusyon sa problema. Maaari kayong mag-sample mula sa mga kandidatong estado na ito at pagkatapos ay suriin ang mga ito gamit ang cost function.

#### Graph &rarr; optimization problem
Ang unang hakbang ng mapping ay isang notation change. Ang sumusunod ay nagpapahayag ng problema sa QUBO notation:

$$
\min_{x\in {0, 1}^n}x^T Q x,
$$

kung saan ang $Q$ ay isang $n\times n$ matrix ng mga real numbers, ang $n$ ay tumutugma sa bilang ng mga node sa inyong graph, ang $x$ ay ang vector ng mga binary variables na ipinakilala sa itaas, at ang $x^T$ ay nagsasaad ng transpose ng vector na $x$.

```
Maximize
 -2*x_0*x_1 - 2*x_0*x_2 - 2*x_0*x_4 - 2*x_1*x_2 - 2*x_2*x_3 - 2*x_3*x_4 + 3*x_0
 + 2*x_1 + 3*x_2 + 2*x_3 + 2*x_4

Subject to
  No constraints

  Binary variables (5)
    x_0 x_1 x_2 x_3 x_4
```
### Optimization problem &rarr; Hamiltonian
Maaari ninyong i-reformulate ang QUBO problem bilang isang **Hamiltonian** (dito, isang matrix na kumakatawan sa enerhiya ng isang sistema):

$$
H_C=\sum_{ij}Q_{ij}Z_iZ_j + \sum_i b_iZ_i.
$$

<details>
<summary>
**Mga hakbang sa reformulation mula sa QAOA problem hanggang sa Hamiltonian**
</summary>

Upang ipakita kung paano ang QAOA problem ay maaaring isulat muli sa paraang ito, palitan muna ang mga binary variables na $x_i$ sa isang bagong set ng mga variables na $z_i\in{-1, 1}$ sa pamamagitan ng

$$
x_i = \frac{1-z_i}{2}.
$$

Dito makikita ninyo na kung ang $x_i$ ay $0$, kung gayon ang $z_i$ ay dapat na $1$. Kapag ang mga $x_i$ ay pinalitan para sa mga $z_i$ sa optimization problem ($x^TQx$), maaaring makuha ang isang equivalent formulation.

$$
x^TQx=\sum_{ij}Q_{ij}x_ix_j \\ =\frac{1}{4}\sum_{ij}Q_{ij}(1-z_i)(1-z_j) \\=\frac{1}{4}\sum_{ij}Q_{ij}z_iz_j-\frac{1}{4}\sum_{ij}(Q_{ij}+Q_{ji})z_i + \frac{n^2}{4}.
$$

Ngayon kung tayo ay magdepina ng $b_i=-\sum_{j}(Q_{ij}+Q_{ji})$, alisin ang prefactor, at ang constant na $n^2$ term, darating tayo sa dalawang equivalent formulations ng parehong optimization problem.

$$
\min_{x\in{0,1}^n} x^TQx\Longleftrightarrow \min_{z\in{-1,1}^n}z^TQz + b^Tz
$$

Dito, ang $b$ ay nakadepende sa $Q$. Tandaan na upang makuha ang $z^TQz + b^Tz$ ay inalis natin ang factor na 1/4 at isang constant offset na $n^2$ na hindi gumaganap ng papel sa optimization.

Ngayon, upang makakuha ng quantum formulation ng problema, itaas ang mga $z_i$ variables sa isang Pauli $Z$ matrix, tulad ng isang $2\times 2$ matrix sa anyo ng

$$
Z_i = \begin{pmatrix}1 & 0 \\ 0 & -1\end{pmatrix}.
$$

Kapag pinalitan ninyo ang mga matrix na ito sa optimization problem sa itaas, makukuha ninyo ang sumusunod na Hamiltonian

$$
H_C=\sum_{ij}Q_{ij}Z_iZ_j + \sum_i b_iZ_i.
$$

*Tandaan din na ang mga $Z$ matrices ay naka-embed sa computational space ng quantum computer, iyon ay, isang Hilbert space na may laki na $2^n\times 2^n$. Samakatuwid, dapat ninyong unawain ang mga terminong tulad ng $Z_iZ_j$ bilang tensor product na $Z_i\otimes Z_j$ na naka-embed sa $2^n\times 2^n$ Hilbert space. Halimbawa, sa isang problemang may limang decision variables, ang termino na $Z_1Z_3$ ay nauunawaan na nangangahulugang $I\otimes Z_3\otimes I\otimes Z_1\otimes I$ kung saan ang $I$ ay ang $2\times 2$ identity matrix.*
</details>

Ang Hamiltonian na ito ay tinatawag na **cost function Hamiltonian**. Mayroon itong property na ang ground state nito ay tumutugma sa solusyon na **nagpapaliit sa cost function $f(x)$**.
Samakatuwid, upang malutas ang inyong optimization problem, kailangan na ninyong ihanda ngayon ang ground state ng $H_C$ (o isang estado na may mataas na overlap dito) sa quantum computer. Pagkatapos, ang pag-sample mula sa estadong ito ay, na may mataas na probabilidad, ay magbubunga ng solusyon sa $min~f(x)$.
Isaalang-alang natin ngayon ang Hamiltonian na $H_C$ para sa **Max-Cut** problem. Hayaang ang bawat vertex ng graph ay maiugnay sa isang qubit sa estado na $|0\rangle$ o $|1\rangle$, kung saan ang halaga ay nagsasaad ng set kung nasaan ang vertex. Ang layunin ng problema ay i-maximize ang bilang ng mga edge na $(v_1, v_2)$ kung saan ang $v_1 = |0\rangle$ at $v_2 = |1\rangle$, o vice-versa. Kung iuugnay natin ang $Z$ operator sa bawat qubit, kung saan

$$
    Z|0\rangle = |0\rangle \qquad Z|1\rangle = -|1\rangle
$$

kung gayon ang isang edge na $(v_1, v_2)$ ay kabilang sa cut kung ang eigenvalue ng $(Z_1|v_1\rangle) \cdot (Z_2|v_2\rangle) = -1$; sa madaling salita, ang mga qubit na nauugnay sa $v_1$ at $v_2$ ay magkaiba. Katulad nito, ang $(v_1, v_2)$ ay hindi kabilang sa cut kung ang eigenvalue ng $(Z_1|v_1\rangle) \cdot (Z_2|v_2\rangle) = 1$. Tandaan na, hindi tayo interesado sa eksaktong qubit state na nauugnay sa bawat vertex, sa halip ang ating pakialam lamang ay kung pareho sila o hindi sa kabuuan ng isang edge. Ang Max-Cut problem ay nangangailangan sa atin na makahanap ng assignment ng mga qubit sa mga vertices nang sa gayon ay ang eigenvalue ng sumusunod na Hamiltonian ay minimized
$$
    H_C = \sum_{(i,j) \in e} Q_{ij} \cdot Z_i Z_j.
$$

Sa madaling salita, ang $b_i = 0$ para sa lahat ng $i$ sa Max-Cut problem. Ang halaga ng $Q_{ij}$ ay nagsasaad ng timbang ng edge. Sa tutorial na ito isinasaalang-alang natin ang isang unweighted graph, iyon ay, $Q_{ij} = 1.0$ para sa lahat ng $i, j$.

In [ ]:
def build_max_cut_paulis(
    graph: rx.PyGraph,
) -> list[tuple[str, list[int], float]]:
    """Convert the graph to Pauli list.

    This function does the inverse of `build_max_cut_graph`
    """
    pauli_list = []
    for edge in list(graph.edge_list()):
        weight = graph.get_edge_data(edge[0], edge[1])
        pauli_list.append(("ZZ", [edge[0], edge[1]], weight))
    return pauli_list


max_cut_paulis = build_max_cut_paulis(graph)
cost_hamiltonian = SparsePauliOp.from_sparse_list(max_cut_paulis, n)
print("Cost Function Hamiltonian:", cost_hamiltonian)

Cost Function Hamiltonian: SparsePauliOp(['IIIZZ', 'IIZIZ', 'ZIIIZ', 'IIZZI', 'IZZII', 'ZZIII'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j])


#### Hamiltonian &rarr; quantum circuit

The Hamiltonian $H_C$ contains the quantum definition of your problem. Now you can create a quantum circuit that will help *sample* good solutions from the quantum computer. The QAOA is inspired by quantum annealing and applies alternating layers of operators in the quantum circuit.

The general idea is to start in the ground state of a known system, $H^{\otimes n}|0\rangle$ above, and then steer the system into the ground state of the cost operator that you are interested in. This is done by applying the operators $\exp\{-i\gamma_k H_C\}$ and $\exp\{-i\beta_k H_m\}$ with angles $\gamma_1,...,\gamma_p$ and $\beta_1,...,\beta_p~$.


The quantum circuit that you generate is **parametrized** by $\gamma_i$ and $\beta_i$, so you can try out different values of $\gamma_i$ and $\beta_i$ and sample from the resulting state.

![Circuit diagram with QAOA layers](../docs/images/tutorials/quantum-approximate-optimization-algorithm/circuit-diagram.svg)


In this case, you will try an example with one QAOA layer that contains two parameters: $\gamma_1$ and $\beta_1$.

In [4]:
circuit = QAOAAnsatz(cost_operator=cost_hamiltonian, reps=2)
circuit.measure_all()

circuit.draw("mpl")

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/7bd8c6d4-f40f-4a11-a440-0b26d9021b53-0.avif" alt="Output of the previous code cell" />

In [5]:
circuit.parameters

ParameterView([ParameterVectorElement(β[0]), ParameterVectorElement(β[1]), ParameterVectorElement(γ[0]), ParameterVectorElement(γ[1])])

### Step 2: Optimize problem for quantum hardware execution

The circuit above contains a series of abstractions useful to think about quantum algorithms, but not possible to run on the hardware. To be able to run on a QPU, the circuit needs to undergo a series of operations that make up the **transpilation** or **circuit optimization** step of the pattern.

The Qiskit library offers a series of **transpilation passes** that cater to a wide range of circuit transformations. You need to make sure that your circuit is **optimized** for your purpose.

Transpilation may involves several steps, such as:

* **Initial mapping** of the qubits in the circuit (such as decision variables) to physical qubits on the device.
* **Unrolling** of the instructions in the quantum circuit to the hardware-native instructions that the backend understands.
* **Routing** of any qubits in the circuit that interact to physical qubits that are adjacent with one another.
* **Error suppression** by adding single-qubit gates to suppress noise with dynamical decoupling.


More information about transpilation is available in our [documentation](/docs/guides/transpile).

The following code transforms and optimizes the abstract circuit into a format that is ready for execution on one of devices accessible through the cloud using the **Qiskit IBM Runtime service**.

In [6]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
print(backend)

# Create pass manager for transpilation
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)

candidate_circuit = pm.run(circuit)
candidate_circuit.draw("mpl", fold=False, idle_wires=False)

<IBMBackend('test_heron_pok_1')>


<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3f28a422-805c-4d3d-b5f6-62539e9133bd-1.avif" alt="Output of the previous code cell" />

### Step 3: Execute using Qiskit primitives

In the QAOA workflow, the optimal QAOA parameters are found in an iterative optimization loop, which runs a series of circuit evaluations and uses a classical optimizer to find the optimal $\beta_k$ and $\gamma_k$ parameters. This execution loop is executed via the following steps:

1. Define the initial parameters
2. Instantiate a new `Session` containing the optimization loop and the primitive used to sample the circuit
3. Once an optimal set of parameters is found, execute the circuit a final time to obtain a final distribution which will be used in the post-process step.

#### Define circuit with initial parameters
We start with arbitrary chosen parameters.

In [7]:
initial_gamma = np.pi
initial_beta = np.pi / 2
init_params = [initial_beta, initial_beta, initial_gamma, initial_gamma]

### Step 2: Optimize problem for quantum hardware execution
Ang circuit sa itaas ay naglalaman ng serye ng mga abstractions na kapaki-pakinabang na isipin tungkol sa quantum algorithms, ngunit hindi posibleng patakbuhin sa hardware. Upang makapagsagawa sa isang QPU, ang circuit ay kailangang dumaan sa serye ng mga operasyon na bumubuo sa **transpilation** o **circuit optimization** step ng pattern.

Ang Qiskit library ay nag-aalok ng serye ng **transpilation passes** na tumutugon sa malawak na hanay ng circuit transformations. Kailangan ninyong siguraduhing ang inyong circuit ay **naka-optimize** para sa inyong layunin.

Ang transpilation ay maaaring magsangkot ng ilang hakbang, tulad ng:

* **Initial mapping** ng mga qubit sa circuit (tulad ng decision variables) sa physical qubits sa device.
* **Unrolling** ng mga instructions sa quantum circuit sa hardware-native instructions na nauunawaan ng backend.
* **Routing** ng anumang mga qubit sa circuit na nakikipag-ugnayan sa physical qubits na katabi ng isa't isa.
* **Error suppression** sa pamamagitan ng pagdaragdag ng single-qubit gates upang pigilan ang ingay sa dynamical decoupling.

Mas maraming impormasyon tungkol sa transpilation ay available sa aming [documentation](/guides/transpile).

Ang sumusunod na code ay nag-transform at nag-optimize ng abstract circuit sa isang format na handa na para sa execution sa isa sa mga devices na accessible sa pamamagitan ng cloud gamit ang **Qiskit IBM Runtime service**.

In [8]:
def cost_func_estimator(params, ansatz, hamiltonian, estimator):
    # transform the observable defined on virtual qubits to
    # an observable defined on all physical qubits
    isa_hamiltonian = hamiltonian.apply_layout(ansatz.layout)

    pub = (ansatz, isa_hamiltonian, params)
    job = estimator.run([pub])

    results = job.result()[0]
    cost = results.data.evs

    objective_func_vals.append(cost)

    return cost

In [9]:
objective_func_vals = []  # Global variable
with Session(backend=backend) as session:
    # If using qiskit-ibm-runtime<0.24.0, change `mode=` to `session=`
    estimator = Estimator(mode=session)
    estimator.options.default_shots = 1000

    # Set simple error suppression/mitigation options
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XY4"
    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"

    result = minimize(
        cost_func_estimator,
        init_params,
        args=(candidate_circuit, cost_hamiltonian, estimator),
        method="COBYLA",
        tol=1e-2,
    )
    print(result)

 message: Return from COBYLA because the trust region radius reaches its lower bound.
 success: True
  status: 0
     fun: -1.6295230263157894
       x: [ 1.530e+00  1.439e+00  4.071e+00  4.434e+00]
    nfev: 26
   maxcv: 0.0


![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3f28a422-805c-4d3d-b5f6-62539e9133bd-1.avif)

### Step 3: Execute using Qiskit primitives
Sa QAOA workflow, ang optimal QAOA parameters ay nahanap sa isang iterative optimization loop, na nagpapatakbo ng serye ng circuit evaluations at gumagamit ng classical optimizer upang makahanap ng optimal na $\beta_k$ at $\gamma_k$ parameters. Ang execution loop na ito ay isinasagawa sa pamamagitan ng sumusunod na mga hakbang:

1. I-define ang mga initial parameters
2. Gumawa ng bagong `Session` na naglalaman ng optimization loop at ang primitive na ginagamit upang mag-sample ng circuit
3. Kapag natagpuan na ang optimal set ng parameters, isagawa ang circuit sa huling pagkakataon upang makakuha ng huling distribution na gagamitin sa post-process step.
#### Define circuit with initial parameters
Nagsisimula tayo sa arbitrarily chosen parameters.

In [10]:
plt.figure(figsize=(12, 6))
plt.plot(objective_func_vals)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/e14ecc92-0.avif" alt="Output of the previous code cell" />

#### Define backend and execution primitive
Gamitin ang **Qiskit Runtime primitives** upang makipag-ugnayan sa IBM&reg; backends. Ang dalawang primitives ay Sampler at Estimator, at ang pagpili ng primitive ay nakadepende sa uri ng measurement na gusto ninyong isagawa sa quantum computer. Para sa minimization ng $H_C$, gamitin ang Estimator dahil ang measurement ng cost function ay simpleng expectation value ng $\langle H_C \rangle$.
#### Run
Ang primitives ay nag-aalok ng iba't ibang [execution modes](/guides/execution-modes) upang mag-schedule ng workloads sa quantum devices, at ang QAOA workflow ay tumatakbo nang iteratively sa isang session.

![Illustration showing the behavior of Single job, Batch, and Session runtime modes.](../docs/images/tutorials/quantum-approximate-optimization-algorithm/runtime-modes.avif)

Maaari ninyong i-plug ang sampler-based cost function sa SciPy minimizing routine upang makahanap ng optimal parameters.

In [11]:
optimized_circuit = candidate_circuit.assign_parameters(result.x)
optimized_circuit.draw("mpl", fold=False, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/2989e76e-4296-4dd8-b065-2b8fced064cf-0.avif" alt="Output of the previous code cell" />

In [12]:
# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10000

# Set simple error suppression/mitigation options
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = "auto"

pub = (optimized_circuit,)
job = sampler.run([pub], shots=int(1e4))
counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_int = {key: val / shots for key, val in counts_int.items()}
final_distribution_bin = {key: val / shots for key, val in counts_bin.items()}
print(final_distribution_int)

{28: 0.0328, 11: 0.0343, 2: 0.0296, 25: 0.0308, 16: 0.0303, 27: 0.0302, 13: 0.0323, 7: 0.0312, 4: 0.0296, 9: 0.0295, 26: 0.0321, 30: 0.031, 23: 0.0324, 31: 0.0303, 21: 0.0335, 15: 0.0317, 12: 0.0309, 29: 0.0297, 3: 0.0313, 5: 0.0312, 6: 0.0274, 10: 0.0329, 22: 0.0353, 0: 0.0315, 20: 0.0326, 8: 0.0322, 14: 0.0306, 17: 0.0295, 18: 0.0279, 1: 0.0325, 24: 0.0334, 19: 0.0295}


### Step 4: Post-process and return result in desired classical format

The post-processing step interprets the sampling output to return a solution for your original problem. In this case, you are interested in the bitstring with the highest probability as this determines the optimal cut. The symmetries in the problem allow for four possible solutions, and the sampling process will return one of them with a slightly higher probability, but you can see in the plotted distribution below that four of the bitstrings are distinctively more likely than the rest.

In [13]:
# auxiliary functions to sample most likely bitstring
def to_bitstring(integer, num_bits):
    result = np.binary_repr(integer, width=num_bits)
    return [int(digit) for digit in result]


keys = list(final_distribution_int.keys())
values = list(final_distribution_int.values())
most_likely = keys[np.argmax(np.abs(values))]
most_likely_bitstring = to_bitstring(most_likely, len(graph))
most_likely_bitstring.reverse()

print("Result bitstring:", most_likely_bitstring)

Result bitstring: [0, 1, 1, 0, 1]


In [14]:
matplotlib.rcParams.update({"font.size": 10})
final_bits = final_distribution_bin
values = np.abs(list(final_bits.values()))
top_4_values = sorted(values, reverse=True)[:4]
positions = []
for value in top_4_values:
    positions.append(np.where(values == value)[0])
fig = plt.figure(figsize=(11, 6))
ax = fig.add_subplot(1, 1, 1)
plt.xticks(rotation=45)
plt.title("Result Distribution")
plt.xlabel("Bitstrings (reversed)")
plt.ylabel("Probability")
ax.bar(list(final_bits.keys()), list(final_bits.values()), color="tab:grey")
for p in positions:
    ax.get_children()[int(p[0])].set_color("tab:purple")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/650875e9-adbc-43bd-9505-556be2566278-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/e14ecc92-0.avif)

Kapag natagpuan na ninyo ang optimal parameters para sa circuit, maaari na ninyong i-assign ang mga parameters na ito at mag-sample ng huling distribution na nakuha gamit ang mga naka-optimize na parameters. Dito dapat gamitin ang *Sampler* primitive dahil ito ang probability distribution ng bitstring measurements na tumutugma sa optimal cut ng graph.

**Tandaan:** Nangangahulugan ito ng paghahanda ng quantum state na $\psi$ sa computer at pagkatapos ay pagsusukat nito. Ang isang measurement ay magko-collapse ng estado sa isang computational basis state - halimbawa, `010101110000...` - na tumutugma sa isang kandidatong solusyon na $x$ sa ating initial optimization problem ($\max f(x)$ o $\min f(x)$ depende sa gawain).

In [15]:
# auxiliary function to plot graphs
def plot_result(G, x):
    colors = ["tab:grey" if i == 0 else "tab:purple" for i in x]
    pos, _default_axes = rx.spring_layout(G), plt.axes(frameon=True)
    rx.visualization.mpl_draw(
        G, node_color=colors, node_size=100, alpha=0.8, pos=pos
    )


plot_result(graph, most_likely_bitstring)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/33135970-8bc4-4fb2-ab87-08726a432ce4-0.avif" alt="Output of the previous code cell" />

And calculate the value of the cut:

In [16]:
def evaluate_sample(x: Sequence[int], graph: rx.PyGraph) -> float:
    assert len(x) == len(
        list(graph.nodes())
    ), "The length of x must coincide with the number of nodes in the graph."
    return sum(
        x[u] * (1 - x[v]) + x[v] * (1 - x[u])
        for u, v in list(graph.edge_list())
    )


cut_value = evaluate_sample(most_likely_bitstring, graph)
print("The value of the cut is:", cut_value)

The value of the cut is: 5


## Part II. Scale it up!

You have access to many devices with over 100 qubits on IBM Quantum&reg; Platform. Select one on which to solve Max-Cut on a 100-node weighted graph. This is a "utility-scale" problem. The steps to build the workflow are followed the same as above, but with a much larger graph.

In [17]:
n = 100  # Number of nodes in graph
graph_100 = rx.PyGraph()
graph_100.add_nodes_from(np.arange(0, n, 1))
elist = []
for edge in backend.coupling_map:
    if edge[0] < n and edge[1] < n:
        elist.append((edge[0], edge[1], 1.0))
graph_100.add_edges_from(elist)
draw_graph(graph_100, node_size=200, with_labels=True, width=1)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/590fe2ce-0.avif" alt="Output of the previous code cell" />

### Step 4: Post-process and return result in desired classical format
Ang post-processing step ay nag-interpret ng sampling output upang magbalik ng solusyon para sa inyong orihinal na problema. Sa kasong ito, interesado kayo sa bitstring na may pinakamataas na probabilidad dahil ito ang tumutukoy sa optimal cut. Ang mga symmetries sa problema ay nagbibigay-daan para sa apat na posibleng solusyon, at ang sampling process ay magbabalik ng isa sa kanila na may bahagyang mas mataas na probabilidad, ngunit makikita ninyo sa plotted distribution sa ibaba na apat sa mga bitstrings ay kapansin-pansing mas malamang kaysa sa iba.

In [18]:
max_cut_paulis_100 = build_max_cut_paulis(graph_100)

cost_hamiltonian_100 = SparsePauliOp.from_sparse_list(max_cut_paulis_100, 100)
print("Cost Function Hamiltonian:", cost_hamiltonian_100)

Cost Function Hamiltonian: SparsePauliOp(['...'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0

#### Hamiltonian &rarr; quantum circuit

In [19]:
circuit_100 = QAOAAnsatz(cost_operator=cost_hamiltonian_100, reps=1)
circuit_100.measure_all()

circuit_100.draw("mpl", fold=False, scale=0.2, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/9693adfc-0.avif" alt="Output of the previous code cell" />

### Step 2: Optimize problem for quantum execution
To scale the circuit optimization step to utility-scale problems, you can take advantage of the high performance transpilation strategies introduced in Qiskit SDK v1.0. Other tools include the new transpiler service with [AI enhanced transpiler passes](/docs/guides/ai-transpiler-passes).

In [20]:
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)

candidate_circuit_100 = pm.run(circuit_100)
candidate_circuit_100.draw("mpl", fold=False, scale=0.1, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3a14e7ad-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/650875e9-adbc-43bd-9505-556be2566278-0.avif)

#### Visualize best cut
Mula sa optimal bit string, maaari na ninyong i-visualize ang cut na ito sa orihinal na graph.

In [21]:
initial_gamma = np.pi
initial_beta = np.pi / 2
init_params = [initial_beta, initial_gamma]

objective_func_vals = []  # Global variable
with Session(backend=backend) as session:
    # If using qiskit-ibm-runtime<0.24.0, change `mode=` to `session=`
    estimator = Estimator(mode=session)

    estimator.options.default_shots = 1000

    # Set simple error suppression/mitigation options
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XY4"
    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"

    result = minimize(
        cost_func_estimator,
        init_params,
        args=(candidate_circuit_100, cost_hamiltonian_100, estimator),
        method="COBYLA",
    )
    print(result)

 message: Return from COBYLA because the trust region radius reaches its lower bound.
 success: True
  status: 0
     fun: -3.9939191365979383
       x: [ 1.571e+00  3.142e+00]
    nfev: 29
   maxcv: 0.0


![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/33135970-8bc4-4fb2-ab87-08726a432ce4-0.avif)

At kalkulahin ang halaga ng cut:

In [22]:
optimized_circuit_100 = candidate_circuit_100.assign_parameters(result.x)
optimized_circuit_100.draw("mpl", fold=False, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/1c432c2e-0.avif" alt="Output of the previous code cell" />

Finally, execute the circuit with the optimal parameters to sample from the corresponding distribution.

In [23]:
# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10000

# Set simple error suppression/mitigation options
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = "auto"


pub = (optimized_circuit_100,)
job = sampler.run([pub], shots=int(1e4))

counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_100_int = {
    key: val / shots for key, val in counts_int.items()
}

## Part II. Scale it up!
Mayroon kayong access sa maraming devices na may higit sa 100 qubits sa IBM Quantum&reg; Platform. Pumili ng isa kung saan lulutasin ang Max-Cut sa isang 100-node weighted graph. Ito ay isang "utility-scale" problem. Ang mga hakbang upang bumuo ng workflow ay sinusunod na pareho tulad sa itaas, ngunit may mas malaking graph.

In [24]:
plt.figure(figsize=(12, 6))
plt.plot(objective_func_vals)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/0fda3611-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/590fe2ce-0.avif)

### Step 1: Map classical inputs to a quantum problem
#### Graph &rarr; Hamiltonian
Una, i-convert ang graph na gusto ninyong lutasin direkta sa isang Hamiltonian na angkop para sa QAOA.

In [25]:
_PARITY = np.array(
    [-1 if bin(i).count("1") % 2 else 1 for i in range(256)],
    dtype=np.complex128,
)


def evaluate_sparse_pauli(state: int, observable: SparsePauliOp) -> complex:
    """Utility for the evaluation of the expectation value of a measured state."""
    packed_uint8 = np.packbits(observable.paulis.z, axis=1, bitorder="little")
    state_bytes = np.frombuffer(
        state.to_bytes(packed_uint8.shape[1], "little"), dtype=np.uint8
    )
    reduced = np.bitwise_xor.reduce(packed_uint8 & state_bytes, axis=1)
    return np.sum(observable.coeffs * _PARITY[reduced])


def best_solution(samples, hamiltonian):
    """Find solution with lowest cost"""
    min_cost = 1000
    min_sol = None
    for bit_str in samples.keys():
        # Qiskit use little endian hence the [::-1]
        candidate_sol = int(bit_str)
        # fval = qp.objective.evaluate(candidate_sol)
        fval = evaluate_sparse_pauli(candidate_sol, hamiltonian).real
        if fval <= min_cost:
            min_sol = candidate_sol

    return min_sol


best_sol_100 = best_solution(final_distribution_100_int, cost_hamiltonian_100)
best_sol_bitstring_100 = to_bitstring(int(best_sol_100), len(graph_100))
best_sol_bitstring_100.reverse()

print("Result bitstring:", best_sol_bitstring_100)

Result bitstring: [1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1]


Next, visualize the cut. Nodes of the same color belong to the same group.

In [26]:
plot_result(graph_100, best_sol_bitstring_100)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/b4a25e28-0.avif" alt="Output of the previous code cell" />

#### Hamiltonian &rarr; quantum circuit

In [27]:
cut_value_100 = evaluate_sample(best_sol_bitstring_100, graph_100)
print("The value of the cut is:", cut_value_100)

The value of the cut is: 124


![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/9693adfc-0.avif)

### Step 2: Optimize problem for quantum execution
Upang i-scale ang circuit optimization step sa utility-scale problems, maaari ninyong samantalahin ang high performance transpilation strategies na ipinakilala sa Qiskit SDK v1.0. Ang iba pang tools ay kinabibilangan ng bagong transpiler service na may [AI enhanced transpiler passes](/guides/ai-transpiler-passes).

In [28]:
# auxiliary function to help plot cumulative distribution functions
def _plot_cdf(objective_values: dict, ax, color):
    x_vals = sorted(objective_values.keys(), reverse=True)
    y_vals = np.cumsum([objective_values[x] for x in x_vals])
    ax.plot(x_vals, y_vals, color=color)


def plot_cdf(dist, ax, title):
    _plot_cdf(
        dist,
        ax,
        "C1",
    )
    ax.vlines(min(list(dist.keys())), 0, 1, "C1", linestyle="--")

    ax.set_title(title)
    ax.set_xlabel("Objective function value")
    ax.set_ylabel("Cumulative distribution function")
    ax.grid(alpha=0.3)


# auxiliary function to convert bit-strings to objective values
def samples_to_objective_values(samples, hamiltonian):
    """Convert the samples to values of the objective function."""

    objective_values = defaultdict(float)
    for bit_str, prob in samples.items():
        candidate_sol = int(bit_str)
        fval = evaluate_sparse_pauli(candidate_sol, hamiltonian).real
        objective_values[fval] += prob

    return objective_values

In [29]:
result_dist = samples_to_objective_values(
    final_distribution_100_int, cost_hamiltonian_100
)

Finally, you can plot the cumulative distribution function to visualize how each sample contributes to the total probability distribution and the corresponding objective value. The horizontal spread shows the range of objective values of the samples in the final distribution. Ideally, you would see that the cumulative distribution function has "jumps" at the lower end of the objective function value axis. This would mean that few solutions with low cost have high probability of being sampled. A smooth, wide curve indicates that each sample is similarly likely, and they can have very different objective values, low or high.

In [30]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
plot_cdf(result_dist, ax, "Eagle device")

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/4381a2b3-0.avif" alt="Output of the previous code cell" />

Kapag natagpuan na ang optimal parameters mula sa pagpapatakbo ng QAOA sa device, i-assign ang mga parameters sa circuit.